In [1]:
import re
from selenium import webdriver
import time
import urllib.request
import os

driver = webdriver.PhantomJS(executable_path="D:/phantomjs.exe")
origin_path = r"./my_review__toefl_vocab.txt"
output_path = r"./VIDEO"

E:\Anaconda3\lib\site-packages\selenium\webdriver\phantomjs\webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


In [2]:
def click_n_geturl(find_idx):
    global html_code
    driver.find_elements_by_class_name(name="msg")[find_idx].click()
    time.sleep(3)
    html_code = driver.page_source
    video_url = re.search(r"http(.*).mp4",html_code)
    video_url = video_url.group()
    img_code = re.findall("img\.iwordshow\.com\/snapshot\/(.*?)\?x-oss-process=style\/w3", html_code)
    img_code = img_code[find_idx]
    img_url = "http://img.iwordshow.com/snapshot/"+img_code
    return video_url, img_url

def extract_inf(word, id=None):
    global html_code
    since = time.time()
    print("retrieving...")
    word_url = "http://www.91dub.com/search/%s?type=search" % word
    driver.get(word_url)
    handles = driver.window_handles
    data = driver.find_elements_by_class_name("msg")
    
    _word = word.replace(" ","_")
    video_name = _word+"_video.mp4"
    img_name = _word+"_img.jpg"
    
    
    if(len(data) == 0): return None
    
    find_idx = None
    bup_idx = None
    find_txt = None
    bup_txt = None

    bup_res = re.search("((.*)\n)([\S\s]*)", data[0].text)
    bup_idx = 0
    bup_txt = bup_res.group()
    if (id == None):
        for idx,blob in enumerate(data):
            #print(blob.text)
            res = re.search("([\S\s]*\n)((.*)\n)([\S\s]*)",blob.text)
            if(res != None):
                try:
                    find_idx = idx
                    find_txt = res.group()
                    video_url, img_url = click_n_geturl(find_idx)
                    print((video_url, img_url))
                    download(video_url, os.path.join(output_path, video_name))
                    download(img_url, os.path.join(output_path, img_name))
                    break
                except:
                    find_idx = None
                    find_txt = None
                    pass

        if(find_idx == None and bup_idx != None):
            try:
                find_idx = bup_idx
                find_txt = bup_txt
                video_url, img_url = click_n_geturl(find_idx)
                download(video_url, os.path.join(output_path, video_name))
                download(img_url, os.path.join(output_path, img_name))
            except:
                find_idx = None
                find_txt = None
                pass
    else:
        find_idx = id
        video_url, img_url = click_n_geturl(find_idx)
        print((video_url, img_url))
        download(video_url, os.path.join(output_path, video_name))
        download(img_url, os.path.join(output_path, img_name))

    if(find_idx == None):
        return None
    
    time_elapsed = time.time() - since
    print('The code run {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print("-----------------------------")
    return video_url, video_name, img_url, img_name, find_txt

def download(url, output_path):
    urllib.request.urlretrieve(url,filename=output_path)

def format_wordlist(origin_path):
    with open(origin_path, encoding="utf-8") as f:
        txt = f.read()
    res = re.findall(u"<div class=\"\"section\"\"\"\"><br>\n([\S\s]*?)(\n)+", txt)
    output_str = ""
    word_list = []
    for blob in res:
        word = re.search(r"\s.*\s", blob[0])
        word = word.group()[1:-1]
        word_list.append(word)
    return word_list

In [3]:
word_list = format_wordlist(origin_path)
anki_list = ""
for word in word_list:
    ret_tuple = extract_inf(word)
    if (ret_tuple == None): continue
    video_url, video_name, img_url, img_name, sentence = ret_tuple
    print(ret_tuple)
    sentence_arr = sentence.split("\n")
    if(len(sentence_arr)==2): sentence_arr.append("/")
    cur_item = word+"~"+video_name+"~"+img_name+"~"+sentence_arr[0]+"~"+sentence_arr[1]+"~"+sentence_arr[2]+"\n"
    anki_list += cur_item
with open("./anki_list.txt", "w") as f:
    f.write(anki_list)

retrieving...
('http://v.iwordshow.com/preview/76f3ef2ea58462ff446d1b5326db13f7.mp4', 'http://img.iwordshow.com/snapshot/CF842C1F825CFED868A68588E5CBB759/601.jpg')
The code run 0m 9s
-----------------------------
('http://v.iwordshow.com/preview/76f3ef2ea58462ff446d1b5326db13f7.mp4', 'arcane_video.mp4', 'http://img.iwordshow.com/snapshot/CF842C1F825CFED868A68588E5CBB759/601.jpg', 'arcane_img.jpg', '太空堡垒卡拉狄加 第一季\narcane scriptural purpose?\n为的是这些离奇古怪的神仙传说吗')
retrieving...
('http://v.iwordshow.com/preview/ea9987fb8cc665cca01024da13cc8318.mp4', 'http://img.iwordshow.com/snapshot/5D9EFDBAD3891F56205A95E320F70873/1309.jpg')
The code run 0m 6s
-----------------------------
('http://v.iwordshow.com/preview/ea9987fb8cc665cca01024da13cc8318.mp4', 'terrific_video.mp4', 'http://img.iwordshow.com/snapshot/5D9EFDBAD3891F56205A95E320F70873/1309.jpg', 'terrific_img.jpg', '前往希腊剧院\nTerrific!Terrific.Great.\n了不起 了不起 太棒啦')
retrieving...
('http://v.iwordshow.com/preview/f5f5d21ec19fe6eadabc22de63fd0094.mp

('http://v.iwordshow.com/preview/19809eb2c3f16f07091ad77430e1219b.mp4', 'http://img.iwordshow.com/snapshot/C44B33EEDB5E8FF149CD5526460C7003/373.jpg')
The code run 0m 5s
-----------------------------
('http://v.iwordshow.com/preview/19809eb2c3f16f07091ad77430e1219b.mp4', 'lurk_video.mp4', 'http://img.iwordshow.com/snapshot/C44B33EEDB5E8FF149CD5526460C7003/373.jpg', 'lurk_img.jpg', "最长的旅程\nI don't lurk.\n才没有")
retrieving...
('http://v.iwordshow.com/preview/4263f53536c807870eeb6030af6e0f16.mp4', 'http://img.iwordshow.com/snapshot/7C8F23E333DD4BA5CDC98E08D7ED3A00/812.jpg')
The code run 0m 5s
-----------------------------
('http://v.iwordshow.com/preview/4263f53536c807870eeb6030af6e0f16.mp4', 'aptitude_video.mp4', 'http://img.iwordshow.com/snapshot/7C8F23E333DD4BA5CDC98E08D7ED3A00/812.jpg', 'aptitude_img.jpg', '分歧者：异类觉醒\nYour aptitude test?\n你的能力倾向测试')
retrieving...
('http://v.iwordshow.com/preview/b12d568e92c232733d5065de47563e02.mp4', 'http://img.iwordshow.com/snapshot/516283BCD56D3047353

The code run 0m 6s
-----------------------------
('http://v.iwordshow.com/preview/ef6651cf3397420b5355b12292454e88.mp4', 'intervene_video.mp4', 'http://img.iwordshow.com/snapshot/1C7855522620BF5E634620006A38AABD/274.jpg', 'intervene_img.jpg', '查理·威尔森的战争\nIntervene?how?\n干涉 怎么干涉啊')
retrieving...
The code run 0m 5s
-----------------------------
('http://v.iwordshow.com/preview/41eb394a34e4582f10655b9d7091d1ef.mp4', 'interdependent_video.mp4', 'http://img.iwordshow.com/snapshot/1622326DB8F18A46BC5A9A6E493D2131/196.jpg', 'interdependent_img.jpg', 'Why Cosmic Evolution Matters: Crash Course Big History #201\nHumanity meanwhile is becoming more ...')
retrieving...
('http://v.iwordshow.com/preview/e046d9e3d4923accaa52aa56ae9c1e55.mp4', 'http://img.iwordshow.com/snapshot/B2F1B13B2AFCCBDE7A255171C2AC16D0/915.jpg')
The code run 0m 5s
-----------------------------
('http://v.iwordshow.com/preview/e046d9e3d4923accaa52aa56ae9c1e55.mp4', 'infringe_video.mp4', 'http://img.iwordshow.com/snapshot/B2F1B

('http://v.iwordshow.com/preview/9454f4c9d0c86e87bfb1696a39952268.mp4', 'http://img.iwordshow.com/snapshot/766A20B4F165BFF129B753034B6C5DC4/123.jpg')
The code run 0m 6s
-----------------------------
('http://v.iwordshow.com/preview/9454f4c9d0c86e87bfb1696a39952268.mp4', 'fascinate_video.mp4', 'http://img.iwordshow.com/snapshot/766A20B4F165BFF129B753034B6C5DC4/123.jpg', 'fascinate_img.jpg', '生活大爆炸 第四季\nFascinate me.\n迷我一下')
retrieving...
('http://v.iwordshow.com/preview/1dbd44929d69dc4784d76319f5d31ebd.mp4', 'http://img.iwordshow.com/snapshot/220A7152FAFCB85D193AB51F76863626/293.jpg')
The code run 0m 7s
-----------------------------
('http://v.iwordshow.com/preview/1dbd44929d69dc4784d76319f5d31ebd.mp4', 'fabulous_video.mp4', 'http://img.iwordshow.com/snapshot/220A7152FAFCB85D193AB51F76863626/293.jpg', 'fabulous_img.jpg', '歌舞青春2\nFabulous,fabulous,fabulous\n惊喜 惊喜 惊喜')
retrieving...
('http://v.iwordshow.com/preview/ffa4220fb2f64dfcff5cd5ecd8fe12a9.mp4', 'http://img.iwordshow.com/snapshot/

('http://v.iwordshow.com/preview/319ad6e1e6e8503bc4c3cb040605b263.mp4', 'http://img.iwordshow.com/snapshot/4921C33E5E78EB25F5DDCD1A00C20B50/41.jpg')
The code run 0m 6s
-----------------------------
('http://v.iwordshow.com/preview/319ad6e1e6e8503bc4c3cb040605b263.mp4', 'estuary_video.mp4', 'http://img.iwordshow.com/snapshot/4921C33E5E78EB25F5DDCD1A00C20B50/41.jpg', 'estuary_img.jpg', 'Exploring Landforms and Bodies of Water for Kids - FreeSchool\nAnother type of coastal landform is the estuary. ...\n另一种海岸地貌是河口。河口是河流与大海相通的地方。')
retrieving...
('http://v.iwordshow.com/preview/1216c1b00360d02ca1839a404597c360.mp4', 'http://img.iwordshow.com/snapshot/C7DC96F7A196FB5EBF3BD1D5F07D6691/587.jpg')
The code run 0m 6s
-----------------------------
('http://v.iwordshow.com/preview/1216c1b00360d02ca1839a404597c360.mp4', 'eradicate_video.mp4', 'http://img.iwordshow.com/snapshot/C7DC96F7A196FB5EBF3BD1D5F07D6691/587.jpg', 'eradicate_img.jpg', '刺客信条\nIf we eradicate free will,we eradicate the ...\n如果我们铲

...

The code run 0m 13s
-----------------------------
('http://v.iwordshow.com/preview/fc7236d8fef2a2d7961c012eb297a028.mp4', 'fallow_video.mp4', 'http://img.iwordshow.com/snapshot/0F25FE0A19D8A1D9C7E524EF359C2CF0/286.jpg', 'fallow_img.jpg', '绯闻女孩 第一季\nFallow.fallow,exactly.fallow.\n卷 卷 没错 卷')
retrieving...
('http://v.iwordshow.com/preview/1f04fb50c057d03d9e5650b57608461b.mp4', 'http://img.iwordshow.com/snapshot/F262771CC720AB57A3C8DE09F00FC539/560.jpg')
The code run 0m 6s
-----------------------------
('http://v.iwordshow.com/preview/1f04fb50c057d03d9e5650b57608461b.mp4', 'alluvial_video.mp4', 'http://img.iwordshow.com/snapshot/F262771CC720AB57A3C8DE09F00FC539/560.jpg', 'alluvial_img.jpg', '耶鲁大学公开课：1600年以来西方社会的流行病\nbuilt at sea level,on alluvial,porous soil.\n海岸线以上的多空土层上的城市')
retrieving...
('http://v.iwordshow.com/preview/add4240b53d6c011ec0c22816dbc1cb8.mp4', 'http://img.iwordshow.com/snapshot/1F43CE85E8A8D8E0CAC2A7738BCDF53F/35.jpg')
The code run 0m 7s
-----------------------------
('ht

('http://v.iwordshow.com/preview/01512b2a5b7322abef279404275d9a34.mp4', 'http://img.iwordshow.com/snapshot/3A564CAD5896A3CAA10F945B3C559A33/41.jpg')
The code run 0m 5s
-----------------------------
('http://v.iwordshow.com/preview/01512b2a5b7322abef279404275d9a34.mp4', 'perplex_video.mp4', 'http://img.iwordshow.com/snapshot/3A564CAD5896A3CAA10F945B3C559A33/41.jpg', 'perplex_img.jpg', '3 rules to spark learning | Ramsey Musallam\nto confuse our students,perplex them,\n混淆学生，使他们陷入困境，')
retrieving...
('http://v.iwordshow.com/preview/e6a0070fbc922e2461ed73dd3972039d.mp4', 'http://img.iwordshow.com/snapshot/3EB10F61192DA254D218D0A58DEB51A0/887.jpg')
The code run 0m 6s
-----------------------------
('http://v.iwordshow.com/preview/e6a0070fbc922e2461ed73dd3972039d.mp4', 'plucky_video.mp4', 'http://img.iwordshow.com/snapshot/3EB10F61192DA254D218D0A58DEB51A0/887.jpg', 'plucky_img.jpg', '极速蜗牛\nThank you,plucky snail.\n谢谢你 大胆的小蜗牛')
retrieving...
('http://v.iwordshow.com/preview/a3d35af78ca19f29696

The code run 0m 5s
-----------------------------
('http://v.iwordshow.com/preview/027aa6b086bfaa38e1f7574a101fb38e.mp4', 'shingle_video.mp4', 'http://img.iwordshow.com/snapshot/A7E7403FD0EF44329DFB50E23F5EFAE3/869.jpg', 'shingle_img.jpg', '耶鲁大学公开课：美国独立战争\nunless you were shingle-by-shingle\n除非你是把印花税征收者房子')
retrieving...
('http://v.iwordshow.com/preview/d61ae0fcc166f88abdebeab56a004d5c.mp4', 'http://img.iwordshow.com/snapshot/18F689B59F7719AF0CAC00B46FD4CF1A/1653.jpg')
The code run 0m 6s
-----------------------------
('http://v.iwordshow.com/preview/d61ae0fcc166f88abdebeab56a004d5c.mp4', 'bundle_video.mp4', 'http://img.iwordshow.com/snapshot/18F689B59F7719AF0CAC00B46FD4CF1A/1653.jpg', 'bundle_img.jpg', '惊天魔盗团2\ntag them and bundle them.\n标记出来')
retrieving...
('http://v.iwordshow.com/preview/2feb71e9a90046fa3fc511c589ae5259.mp4', 'http://img.iwordshow.com/snapshot/13893AA4A7E19C4D5C41AC8CC360C11D/812.jpg')
The code run 0m 7s
-----------------------------
('http://v.iwordshow.com/preview/

('http://v.iwordshow.com/preview/592b94801fd464068bde625310466940.mp4', 'http://img.iwordshow.com/snapshot/C8084876DE0C3DDFE8BC44AA44A4547E/1184.jpg')
The code run 0m 6s
-----------------------------
('http://v.iwordshow.com/preview/592b94801fd464068bde625310466940.mp4', 'palatable_video.mp4', 'http://img.iwordshow.com/snapshot/C8084876DE0C3DDFE8BC44AA44A4547E/1184.jpg', 'palatable_img.jpg', '嚎叫\nmore palatable for good taste.\n才是大快朵颐的精神食粮的人们')
retrieving...
('http://v.iwordshow.com/preview/06645e1119448fe6ab94f014df7c226d.mp4', 'http://img.iwordshow.com/snapshot/8FD0DE91EF5C3ACE3D1E9B8DF30C8EA9/20.jpg')
The code run 0m 6s
-----------------------------
('http://v.iwordshow.com/preview/06645e1119448fe6ab94f014df7c226d.mp4', 'mistrust_video.mp4', 'http://img.iwordshow.com/snapshot/8FD0DE91EF5C3ACE3D1E9B8DF30C8EA9/20.jpg', 'mistrust_img.jpg', 'Shakespearean dating tips - Anthony John Peters\nmistrust,\n欺诈')
retrieving...
('http://v.iwordshow.com/preview/9fca1ad44a34234e9beeb68aeea55056.mp

('http://v.iwordshow.com/preview/2f1ea08cf37f53708433db214baa2880.mp4', 'http://img.iwordshow.com/snapshot/AF362F20C3B7CB3BBE73D29BAAD25CE2/317.jpg')
The code run 0m 9s
-----------------------------
('http://v.iwordshow.com/preview/2f1ea08cf37f53708433db214baa2880.mp4', 'cleanse_video.mp4', 'http://img.iwordshow.com/snapshot/AF362F20C3B7CB3BBE73D29BAAD25CE2/317.jpg', 'cleanse_img.jpg', '林中小屋\nCleanse them.Cleanse the world of their ignorance ...\n净化他们 净化他们的无知与罪恶')
retrieving...
('http://v.iwordshow.com/preview/0b701ca377a96446829eeedde14946a0.mp4', 'http://img.iwordshow.com/snapshot/0E68F956AE95E8A3772F32CE3ED5F22F/933.jpg')
The code run 0m 7s
-----------------------------
('http://v.iwordshow.com/preview/0b701ca377a96446829eeedde14946a0.mp4', 'proceeding_video.mp4', 'http://img.iwordshow.com/snapshot/0E68F956AE95E8A3772F32CE3ED5F22F/933.jpg', 'proceeding_img.jpg', '火星救援\nProceeding with the count.\n进入倒数')
retrieving...
('http://v.iwordshow.com/preview/33d0dd1a082d609f27647e0121b149a1.

In [6]:
extract_inf("fecund")

retrieving...
